## Libraries

In [69]:
import numpy as np
import pulp as pl
import random

## Src

In [70]:
def generate_vrptw_data(
    num_customers,
    max_demand,
    max_travel_time,
    max_service_time,
    max_time_window,
    max_cost,
):
    demands = np.array(
        [0] + [random.randint(1, max_demand) for _ in range(num_customers)]
    )

    travel_times = np.zeros((num_customers + 1, num_customers + 1), dtype=int)
    for i in range(num_customers + 1):
        for j in range(i + 1, num_customers + 1):
            travel_times[i, j] = travel_times[j, i] = random.randint(1, max_travel_time)

    costs = np.zeros((num_customers + 1, num_customers + 1), dtype=int)
    for i in range(num_customers + 1):
        for j in range(i + 1, num_customers + 1):
            costs[i, j] = costs[j, i] = random.randint(1, max_cost)

    time_windows = np.array(
        [(0, max_time_window)]
        + [
            (
                random.randint(0, max_time_window // 2),
                random.randint(max_time_window // 2, max_time_window),
            )
            for _ in range(num_customers)
        ]
    )

    service_times = np.array(
        [0] + [random.randint(1, max_service_time) for _ in range(num_customers)]
    )

    return demands, travel_times, costs, time_windows, service_times



## Constants

In [71]:
M = 1000  # Grande constante pour les contraintes de tempsQ

max_demand = 5
max_travel_time = 30
max_service_time = 15
max_time_window = 100
N = 10  # Nombre de points de collecte (sans compter le dépôt)
num_camions = 2
camion_capacity = 15
max_cost = 50

demands, travel_times, costs, time_windows, service_times = generate_vrptw_data(
    N,
    max_demand,
    max_travel_time,
    max_service_time,
    max_time_window,
    max_cost,
)
print("Demands:", demands)
print("Travel Times:\n", travel_times)
print("Costs:\n", costs)
print("Time Windows:", time_windows)
print("Service Times:", service_times)

Demands: [0 2 1 5 1 1 2 2 5 1 4]
Travel Times:
 [[ 0 24 29 20  9  9 29  6 16 13 23]
 [24  0 22  3  9  1 24  2 28 25 15]
 [29 22  0 12 17 25 24  2 28 19 16]
 [20  3 12  0  2  2 16  7  8  3  4]
 [ 9  9 17  2  0  4 12  6 27 20  2]
 [ 9  1 25  2  4  0  6  9 16 23 14]
 [29 24 24 16 12  6  0 13  9 14 18]
 [ 6  2  2  7  6  9 13  0 24 19 18]
 [16 28 28  8 27 16  9 24  0 29 27]
 [13 25 19  3 20 23 14 19 29  0  2]
 [23 15 16  4  2 14 18 18 27  2  0]]
Costs:
 [[ 0  6 32 47  1  4 24 47 49 34  6]
 [ 6  0 33 33 29 38 34 38 23 22 35]
 [32 33  0  8 38  3 40  8  2 18  3]
 [47 33  8  0 34 30 18 43 14 45 34]
 [ 1 29 38 34  0 34  2 41 44 37  4]
 [ 4 38  3 30 34  0 15 32 43 45 45]
 [24 34 40 18  2 15  0  8 23 33 39]
 [47 38  8 43 41 32  8  0  6 35 46]
 [49 23  2 14 44 43 23  6  0 48  1]
 [34 22 18 45 37 45 33 35 48  0 15]
 [ 6 35  3 34  4 45 39 46  1 15  0]]
Time Windows: [[  0 100]
 [  0  59]
 [ 12  94]
 [ 37  82]
 [ 20  71]
 [ 16  72]
 [ 42  92]
 [ 42  57]
 [ 27  89]
 [ 28  95]
 [ 36  96]]
Service Times:

Modèle de minimisation de coût

In [72]:
vrptw_model = pl.LpProblem("Optimisation_Collecte_Dechets", pl.LpMinimize)

Variables de décision

In [73]:
# x = pl.LpVariable.dicts("x", ((i, j, k) for i in range(N+1) for j in range(N+1) for k in range(num_camions)), cat='Binary')
# s = pl.LpVariable.dicts("s", (i for i in range(N+1)), lowBound=0, cat='Continuous')

In [74]:
x = pl.LpVariable.dicts(
    "x",
    ((i, j, k) for i in range(N + 1) for j in range(N + 1) for k in range(num_camions)),
    cat="Binary",
)
s = pl.LpVariable.dicts(
    "s",
    ((i, k) for i in range(N + 1) for k in range(num_camions)),
    lowBound=0,
    cat="Continuous",
)

Fonction objectif : minimiser la distance totale parcourue

In [76]:
vrptw_model += (
    pl.lpSum(
        costs[i][j] * x[i, j, k]
        for i in range(N + 1)
        for j in range(N + 1)
        for k in range(num_camions)
    ),
    "TotalCost",
)

## constraints

1. Each customer is visited exactly once

In [77]:
for i in range(1, N + 1):
    vrptw_model += (
        pl.lpSum(
            x[i, j, k] for j in range(N + 1) for k in range(num_camions)
        )
        == 1,
        f"VisitOnce_{i}",
    )

2. Vehicle capacity constraint

In [78]:
for k in range(num_camions):
    vrptw_model += (
        pl.lpSum(
            demands[i] * x[i, j, k]
            for i in range(N + 1)
            for j in range(N + 1)
        )
        <= camion_capacity,
        f"Capacity_{k}",
    )

3. Flow conservation constraint (leave each node after entering it)

In [79]:
for k in range(num_camions):
    for i in range(N + 1):
        vrptw_model += (
            pl.lpSum(x[i, j, k] for j in range(N + 1))
            == pl.lpSum(x[j, i, k] for j in range(N + 1)),
            f"FlowConservation_{i}_{k}",
        )


4. Start and end at the depot

In [80]:
for k in range(num_camions):
    vrptw_model += (
        pl.lpSum(x[0, j, k] for j in range(1, N + 1)) == 1,
        f"StartAtDepot_{k}",
    )
    vrptw_model += (
        pl.lpSum(x[j, 0, k] for j in range(1, N + 1)) == 1,
        f"EndAtDepot_{k}",
    )

5. Time window constraints

In [81]:
for k in range(num_camions):
    for i in range(N + 1):
        for j in range(1, N + 1):  # skip depot as destination
            if i != j:
                vrptw_model += (
                    s[i, k] + travel_times[i][j] + service_times[i] <= s[j, k] + (1 - x[i, j, k]) * 1e6
                ), f"TimeWindow_{i}_{j}_{k}"

    for i in range(1, N + 1):  # skip depot
        vrptw_model += (time_windows[i][0] <= s[i, k]), f"TimeWindowStart_{i}_{k}"
        vrptw_model += (s[i, k] <= time_windows[i][1]), f"TimeWindowEnd_{i}_{k}"


---

## Exact method

In [83]:
vrptw_model.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/cuphead/Projects/VRPTW-Meta-Heuristique/venv/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/bb9223242daf4682bc5d603d92627365-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/bb9223242daf4682bc5d603d92627365-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 283 COLUMNS
At line 2570 RHS
At line 2849 BOUNDS
At line 3092 ENDATA
Problem MODEL has 278 rows, 264 columns and 1560 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 10 - 0.00 seconds
Cgl0003I 10 fixed, 0 tightened bounds, 182 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 172 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 172 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 172 strengthened rows, 0 substitutions
Cgl000

1

In [85]:
if vrptw_model.status == pl.LpStatusOptimal:
    print("Optimal solution found with total cost:", pl.value(vrptw_model.objective))
    routes = []
    for k in range(num_camions):
        route = [0]  # Start at depot
        current_location = 0
        while True:
            next_location = None
            for j in range(N + 1):
                if pl.value(x[current_location, j, k]) == 1:
                    route.append(j)
                    next_location = j
                    break
            if next_location == 0 or next_location is None:
                break
            current_location = next_location
        routes.append(route)
    print("Routes:", routes)
else:
    print("No optimal solution found.")

Optimal solution found with total cost: 10.0
Routes: [[0, 4, 0], [0, 5, 0]]


---

## Meta-heuristique

In [65]:
depot = 0  # Dépôt

# Paramètres du Recuit Simulé
initial_temperature = 1000
cooling_rate = 0.95
stopping_temperature = 1e-8
max_iterations = 1000

# Fonction de coût
def calculate_cost(solution):
    total_cost = 0
    for route in solution:
        route_cost = 0
        for i in range(len(route) - 1):
            route_cost += costs[route[i]][route[i + 1]]
        total_cost += route_cost
    return total_cost

# Fonction pour générer une solution initiale
def generate_initial_solution():
    solution = []
    nodes = list(range(1, N + 1))
    for _ in range(num_camions):
        route = [depot]
        load = 0
        while nodes and load + demands[nodes[0]] <= capacity_camion:
            node = nodes.pop(0)
            route.append(node)
            load += demands[node]
        route.append(depot)
        solution.append(route)
    return solution

# Fonction pour générer une solution voisine
def generate_neighbor(solution):
    neighbor = [route[:] for route in solution]
    i, j = random.randint(0, num_camions - 1), random.randint(1, len(neighbor[0]) - 2)
    if neighbor[i][j] != depot:
        neighbor[i][j], neighbor[(i + 1) % num_camions][j] = neighbor[(i + 1) % num_camions][j], neighbor[i][j]
    return neighbor

# Fonction pour exécuter le Recuit Simulé
def simulated_annealing():
    current_solution = generate_initial_solution()
    current_cost = calculate_cost(current_solution)
    best_solution = current_solution
    best_cost = current_cost
    temperature = initial_temperature

    while temperature > stopping_temperature:
        for _ in range(max_iterations):
            neighbor = generate_neighbor(current_solution)
            neighbor_cost = calculate_cost(neighbor)
            cost_diff = neighbor_cost - current_cost

            if cost_diff < 0 or np.exp(-cost_diff / temperature) > random.random():
                current_solution = neighbor
                current_cost = neighbor_cost
                if current_cost < best_cost:
                    best_solution = current_solution
                    best_cost = current_cost

        temperature *= cooling_rate
    return best_solution, best_cost

# Exécution de l'algorithme
best_solution, best_cost = simulated_annealing()
print("Meilleure solution:", best_solution)
print("Coût total:", best_cost)

Meilleure solution: [[0, 1, 2, 3, 8, 0], [0, 5, 6, 7, 4, 9, 0]]
Coût total: 57
